In [53]:
from pathlib import Path

from rosbags.highlevel import AnyReader
from rosbags.typesys import Stores, get_typestore

import struct
import pandas as pd
import numpy as np
import csv

#### Extract Sensor Data
##### '/livox/imu' publishes sensor_msgs/IMU message
###### Raw Msg Def: Header, quaterion orientation, orientation cov, angular velocity, velocity cov, linear acceleration, linear acceleration covariance
##### '/livox/lidar' publishes sensor_msgs/PointCloud message
The message is organized as follows {'name: (offset, datatype)}
###### {'x': (0, 7), 'y': (4, 7), 'z': (8, 7), 'intensity': (12, 7), 'tag': (16, 2), 'line': (17, 2), 'timestamp': (18, 8)}
###### uint8 UINT8 = 2, uint8 FLOAT32 = 7, uint8 FLOAT64 = 8


In [66]:
bagpath = Path('no_motion')

# Create a type store to use if the bag has no message definitions.
typestore = get_typestore(Stores.ROS2_HUMBLE)

imu_data = []
cloud_data = []

# Create reader instance and open for reading.
with AnyReader([bagpath], default_typestore=typestore) as reader:
    connections = [x for x in reader.connections if x.topic == '/imu_raw/Imu']
    for connection, timestamp, rawdata in reader.messages(connections=connections):
        msg = reader.deserialize(rawdata, connection.msgtype)

        if connection.msgtype == 'sensor_msgs/msg/Imu':
            orientations = [msg.orientation.x, msg.orientation.y, msg.orientation.z]
            ang_velocity = [msg.angular_velocity.x, msg.angular_velocity.y, msg.angular_velocity.z]
            lin_acceleration = [msg.linear_acceleration.x, msg.linear_acceleration.y, msg.linear_acceleration.z]

            data = [timestamp, orientations, ang_velocity, lin_acceleration]
            imu_data.append(data)

        elif connection.msgtype == 'sensor_msgs/msg/PointCloud2':
            x_all = []
            y_all = []
            z_all = []
            intensity_all = []
            tag_all = []
            line_all = []

            points = msg.data#tolist()
            N = int((points.shape[0])/26)
            points = points.reshape((N,26))
            width = msg.width
            height = msg.height
            field = msg.fields

            # Extract data frin field
            field_names = {field.name: (field.offset, field.datatype) for field in msg.fields}
            struct_fmt = "ffffBB"
            f = struct.calcsize('f')
            d=struct.calcsize('d')
            B = struct.calcsize('B')
            for row in points:
                x,y,z,intensity,tag,line = struct.unpack(struct_fmt, bytes(row[0:-8]))
                x_all.append(x)
                y_all.append(y)
                z_all.append(z)
                intensity_all.append(intensity)
                tag_all.append(tag)
                line_all.append(line)


            

            cloud_data.append([x_all,y_all,z_all,intensity_all,tag_all,line_all])
        #  print(msg.header.frame_id)


In [24]:
with open('imu.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for item in imu_data:
        writer.writerow([item])

In [67]:
with open('point_cloud.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for item in cloud_data:
        writer.writerow([item])

##